# Install required dependencies

In [1]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -r /kaggle/input/waterminder-v26/WaterMinder_V26/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [tamm]6/7 [tamm]ltools]
Note: you may need to restart the kernel to use updated packages.


# One-Time Setup
- Run this cell only once at the beginning of your Kaggle session. Its job is to copy the toolkit
- from the read-only /kaggle/input to the writable /kaggle/working directory and fix its package structure.

In [3]:
# Block 1: Run this only once per session
import os
import shutil
from pathlib import Path

print("Starting one-time setup...")

# 1. Define source and destination paths
source_path = Path("/kaggle/input/waterminder-v26/WaterMinder_V26")
destination_path = Path("/kaggle/working/WaterMinder")

# 2. Copy the entire toolkit to the new writable directory
if destination_path.exists():
    print(f"'{destination_path.name}' already exists. Removing it.")
    shutil.rmtree(destination_path)
print(f"Copying toolkit to '{destination_path}'...")
shutil.copytree(source_path, destination_path)

# 3. Create the necessary '__init__.py' files to make it a package
print("Creating package files...")
(destination_path / "__init__.py").touch()
(destination_path / "examples" / "__init__.py").touch()

# Check if the 'utils' subdirectory exists and add an __init__.py there too
if (destination_path / "examples" / "utils").is_dir():
    (destination_path / "examples" / "utils" / "__init__.py").touch()

print("\n✅ One-time setup complete. You can now run the main code block.")

Starting one-time setup...
Copying toolkit to '/kaggle/working/WaterMinder'...
Creating package files...

✅ One-time setup complete. You can now run the main code block.


# Check The Dependencies Installed or Not 

- Ensure Python searches the current working directory for modules so the example scripts can be found and check that all depedencies are installed.

In [4]:
from pathlib import Path
import sys
import os
import logging
sys.path.append(os.path.abspath(""))
sys.path.append(os.path.abspath("/kaggle/working/WaterMinder"))

if not logging.getLogger().hasHandlers():
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Check all dependent packages are installed
import importlib.util
dependent_packages = ["coremltools", "torch", "sentencepiece", "tamm", "examples", "export"]
missing_packages = []
for package_name in dependent_packages:
    spec = importlib.util.find_spec(package_name)
    if spec is None:
        print(f"❌ Package '{package_name}' is NOT installed or cannot be found.")
        missing_packages.append(package_name)

# Check assets
dependent_assets = {
    "weights_template.bin", "checkpoint_spec.yaml", "draft.mil",
    "base-model-config.json", "base-model.pt", "draft-model-config.json", "draft-model.pt",
    "tokenizer-config.json", "tokenizer.model",
}
missing_assets = dependent_assets - set([path.name for path in Path("/kaggle/working/WaterMinder/assets").rglob("*")])
for asset_name in missing_assets:
    print(f"❌ Asset '{asset_name}' cannot be found.")

if not missing_packages and not missing_assets:
    print("✅ All set")

✅ All set


# Test Model

In [5]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("I'm new to the app. Can you walk me through the initial setup, and then explain how to customize the default cups and manually change the daily goal the app sets for me?")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

Welcome to WaterMinder! The app is designed to help you track your daily water intake and stay properly hydrated.

## Initial Setup

1.  **Open the App**: Launch WaterMinder on your iPhone.
2.  **Create an Account**: Tap the **+** icon at the bottom of the screen.
3.  **Enter Your Information**: On the "Sign Up" screen, enter your full name, email address, and create a password.
4.  **Configure Your Profile**: On the "Edit Profile" screen, you can change your profile name, choose a profile picture, and select your weight and activity level.

---

## Customizing Cups

You can customize any of the default cups by going to **Settings > Cups**.

1.  **Select a Cup**: Tap on the default cup you want to change.
2.  **Customize its Properties**: On the "Edit Cup" screen, you can change the:
    * **Name**: Give it a descriptive label, like "My Work Bottle."
    * **Drink Amount**: Change the volume to a precise amount.
    * **Drink Type**: Change the liquid (e.g., coffee, tea, juice).
    * **Icon and Color**: Choose a new symbol and color to represent it.
3.  **Save Your Changes**: Once you're done, scroll to the bottom and tap the **Save** button.

---

## Changing Your Daily Goal

You can set your own personalized goal by going to **Settings > Cups > Advanced**.

1.  **Tap on Advanced**: In the Cups menu, tap the **Advanced** option.
2.  **Enter Your Goal**: On the "Edit Cup" screen, tap on the **Goal** field.
3.  * **Type Your Goal**: Type in the exact amount you wish to drink daily (e.g., change the 8 oz cup's goal to 30 oz).
4.  * **Save Your Changes**: After entering your new goal, scroll to the bottom and tap the **Save** button.<turn_end>

In [6]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("Adding WaterMinder Widgets to Your Home Screen")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

You can add WaterMinder widgets to your home screen to make tracking your water intake even easier. Your device needs to be on **iOS 14 or above**.

## How to Add a WaterMinder Widget

1.  **Tap and hold** an empty space on your home screen to enter "jiggle mode."
2.  Tap the plus sign **(+)** in the top corner of the screen.
3.  In the search box, type **WaterMinder** and select it.
4.  You'll see all the available widgets. Choose the one you want and tap **Add Widget**.

---

## Widget Options

WaterMinder offers a variety of widgets to suit your needs:

* **Current Hydration Simple**: A simple visual of your progress.
* **Current Hydration with Avatar**: Adds your profile image to the graphic.
* **Current Hydration with a Weekly Graph**: Shows today's progress and your intake history for the past week.
* **Current Hydration with Cups**: A large widget showing your progress and favorite cups. Tapping a cup automatically logs the drink.
* **Current Hydration with a Weekly Graph (Large)**: An enlarged version of the graph widget, useful for quick overviews.
* **Detailed Hydration (Small)**: A compact widget displaying today's progress, a weekly graph, and other stats.
* **Detailed Hydration (Medium)**: A larger version of the detailed widget, providing more information at a glance.
* **Detailed Hydration (Large)**: The largest widget, offering a comprehensive view of your intake data, including a large graph and additional stats.

To **customize** a widget, tap and hold it, then select **Edit Widget**. To **remove** it, tap and hold, then select **Remove Widget**.<turn_end>

In [7]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("How do the different reminder types work in WaterMinder?")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

WaterMinder's reminders are designed to help you drink water regularly throughout the day. There are three main reminder types you can choose from in the app's settings:

* **Static**: This is the default setting. It allows you to set reminders for specific times of the day (e.g., 8:00 AM, 9:30 AM) or create your own custom reminders.
* **Interval**: This mode sends you reminders at a regular interval (e.g., every 60 minutes) between a start and end time that you define.
* **Water Level**: This mode ties your reminders to progress. It sets percentage goals for certain times of the day (e.g., reach 30% of your goal by 9:30 AM). If you're on track, the reminders will pause until the next time period begins.<turn_end>

In [8]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("How do you delete drinks logged by mistake?")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

It's easy to remove a drink that you've logged by mistake in WaterMinder. Here's a step-by-step guide.

## How to Delete a Drink from Your History

1.  **Go to the History Tab**: Open the WaterMinder app and tap the **History** icon at the bottom of the screen.
2.  **Find the Mistaken Entry**: You'll see a list of all the drinks you've logged for the current day. To find an entry from a previous day, use the arrows at the top to navigate to the correct date.
3.  **Swipe Left to Delete**: Once you've located the incorrect entry, simply swipe left across that row.
4.  **Confirm Deletion**: A red **Delete** button will appear. Tap it to permanently remove the logged drink.

**Quick Tip**: If you realize you made a mistake immediately after logging a drink, look for an **"Undo"** button that often appears at the top of the main screen for a few seconds. Tapping it will instantly reverse the last action.<turn_end>

In [9]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("Can you give me a detailed guide on how to customize cups in WaterMinder")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

You can customize any cup in WaterMinder, including changing its name, size, drink type, and appearance. Here’s a detailed breakdown of how to do it.

## Customization Steps

1.  **Access Cup Settings**: Go to **Settings > Cups** to access all customization options.
2.  **Select a Cup**: Tap on the specific cup you want to change.
3.  **Customize Name**: Tap on **Name**, enter a new name for the cup, and tap **Done**.
4.  **Change Drink Type & Hydration Impact**: Tap on **Drink Type**. You can select from options like Coffee, Tea, Juice, etc. Each drink has a different **Hydration Impact** (e.g., water is 1.0, coffee is 0.9). Select the drink type you want and tap **Done**.
5.  **Adjust Size & Save**: Tap on **Drink Amount**, enter the new size for the cup, and tap **Save**.
6.  **Modify Appearance**: To change the icon and its color, tap on **Icon** and **Color** respectively and make your selections.
7.  **Configure Other Options**: On this screen, you can also manage settings for Siri shortcuts and the Siri Watch Face.
8.  **Save Everything**: When you're finished, tap the main **Save** button at the bottom of the screen.<turn_end>

In [10]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("I want to Details of all Features and what are the main Feature of the WaterMinder app")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

WaterMinder is a popular and highly-rated app designed to help you track your daily water intake and stay properly hydrated. The app is based on the idea that drinking enough water is crucial for your health, energy, and even skin appearance. In this detailed breakdown, we'll cover all the key features of WaterMinder, explain their purpose, and highlight the main features that make the app so effective.

## 1. **Personalized Daily Goal Setting**

This is the most important feature of WaterMinder. When you first set up the app, WaterMinder calculates a suggested daily water intake goal based on your body weight, activity level, and even the local weather. You can also set your own custom goal. The app's interface is designed to make reaching your goal easy and intuitive.

---

## 2. **Easy Tracking of Drinks**

WaterMinder makes tracking your water intake simple. You can log drinks by taking a picture of your cup, entering the amount manually, or using the app's barcode scanner. The app has pre-defined cup sizes but also lets you create custom cups for various beverages like coffee, tea, and juice.

---

## 3. **Visual Progress and Widgets**

The app's interface is clean and visual, with a large, easy-to-read progress bar that shows your daily progress at a glance. WaterMinder also allows you to add widgets to your iPhone's Home Screen and Lock Screen, providing quick updates and progress checks without needing to open the app.

---

## 4.  **Apple Health Integration**

WaterMinder seamlessly integrates with your Apple Health data. It syncs your water intake logs and progress automatically, allowing you to view your drinking habits and achievements directly in the Health app.

---

## 5.  **Achievements and Badges**

To keep you motivated, WaterMinder includes a fun and gamified element. The app awards you with achievements and badges for reaching certain milestones, such as logging your first 1,000ml, hitting your daily goal, or staying on track for a week.

---

## 6.  **Detailed Analysis and Reports**

WaterMinder goes beyond just tracking your intake. The app provides detailed reports on your drinking habits, including graphs and charts that show your progress over time. It also categorizes your drinks (e

In [13]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("How do I install WaterMinder on my Apple Watch?")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

To use WaterMinder on your Apple Watch, you first need to have the app installed on your paired iPhone. You can then install the watch app using one of two easy methods.

## Method 1: Using Your iPhone

This is the most common way to manage your watch apps.

1.  On your iPhone, open the **Watch** app.
2.  Make sure you are on the **My Watch** tab.
3.  Scroll down to the **Available Apps** list.
4.  Locate WaterMinder and tap the **Install** button next to it. The app will then appear on your watch's Home Screen.

---

## Method 2: Using the App Store on Your Apple Watch

You can also download the app directly from your watch.

1.  Press the Digital Crown on your Apple Watch and tap the **App Store** icon.
2.  Tap the **Search** bar and use Scribble or Dictation to find "WaterMinder."
3.  Tap the **Get** button (or the cloud icon) to download the app directly to your watch.<turn_end>

In [14]:
from WaterMinder.examples.generate import generate_content, GenerationConfiguration
from WaterMinder.examples.messages import Message

output = generate_content(
    [[
        Message.from_system("You are a helpful assistant providing information about the WaterMinder app."),
        Message.from_user("What are the options for installing the WaterMinder app on my Apple Watch?")
    ]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=512),
    checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/adapter-epoch5.pt",
    draft_checkpoint="/kaggle/input/wm_adapter_model/pytorch/new_model_with_new_data/1/draft-model-final.pt"
)

# To display the output
from IPython.display import Markdown, display
display(Markdown(output[0].response))

You have two main options for installing the WaterMinder app on your Apple Watch.

## Option 1: Download from the App Store

Use this method if you're installing the app for the first time on either device.

1.  Tap the **Watch app** on your iPhone.
2.  Go to the **App Store** section and tap the **Search** tab.
3.  Type "WaterMinder" in the search bar and then download the app.

---

## Option 2: Sync From Your iPhone

If you already have WaterMinder on your phone, use this method to show it on your watch.

1.  Tap the **Watch app** on your iPhone.
2.  On the **My Watch** tab, scroll down and tap on **WaterMinder**.
3.  Make sure the **"Show App on Apple Watch"** toggle is turned on (green).

**Troubleshooting**: If you can't see WaterMinder in the list of available apps, try restarting both your iPhone and your Apple Watch.<turn_end>